In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import io #Used as buffer

In [2]:
path= Path(f'../../data/cluster_results')

In [3]:
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['std'] = pd.read_csv(f).std().iloc[0] ##added line for std, remove this will remove std from table
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    #print(name)
    df['freq'] = float(name[4:].split('n')[0])
    #print(df['freq'])
    df['method'] = f.name.split('_')[0]+'_'+f.name.split('_')[1]
    #df['method'] = f.name.split('_')[0]
    #print(df['method'])
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0

In [4]:
m = measure_name[index_measure]
files  = list(f for f in path.glob(f'exp*/*/freq*nn_p3nn_s2nn_tw5*bs4*/*{m}.csv') if 'Closest' not in f.name)
# files += list(path.glob(f'exp*/*/*nn_p3/*{m}.csv'))
files += [f for f in path.glob(f'exp*/*/freq*nn_p3nn_s2nn_tw5*bs4*/*{m}.csv') if 'noNN' in f.name] #f'exp*/*/*nn_p3/*{m}.csv'  , 
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=['experiment','function','freq'],  columns='method') #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [5]:
#to add std to table
final_data = data[m].copy()

for col in data[m].columns:
    final_data[col] = data[m][col].apply(lambda x: f'{x:,.2f} (±') + data['std'][col].apply(lambda x: f'{x:,.2f})')
final_data

method                                      NNdrop_Random  \
experiment function   freq                                  
exp1       rastrigin  0.5                   24.14 (±3.93)   
                      1.0                   21.33 (±3.68)   
                      4.0                   19.29 (±6.28)   
           rosenbrock 0.5        680,083.62 (±468,075.91)   
                      1.0        253,692.23 (±151,509.51)   
                      4.0        129,485.47 (±156,675.88)   
           sphere     0.5                   18.31 (±4.67)   
                      1.0                   13.27 (±6.87)   
                      4.0                   10.94 (±8.66)   
exp2       rastrigin  0.5                    2.55 (±0.37)   
                      1.0                    1.82 (±0.17)   
                      4.0                    0.98 (±0.12)   
           rosenbrock 0.5                 260.34 (±69.03)   
                      1.0                 130.58 (±35.87)   
                      4.0                  20.95 (±10.89)   
           sphere     0.5                    0.34 (±0.13)   
                      1.0                    0.19 (±0.05)   
                      4.0                    0.03 (±0.01)   
exp3       rastrigin  0.5                    8.66 (±2.86)   
                      1.0                   15.77 (±4.33)   
                      4.0                   27.09 (±1.83)   
           rosenbrock 0.5              3,942.05 (±758.65)   
                      1.0              4,166.39 (±112.07)   
                      4.0              4,089.81 (±775.64)   
           sphere     0.5                    5.16 (±0.18)   
                      1.0                    5.28 (±0.11)   
                      4.0                    5.41 (±0.08)   
exp4       rastrigin  0.5                   77.21 (±6.12)   
                      1.0                   56.20 (±3.41)   
                      4.0                   35.96 (±3.95)   
           rosenbrock 0.5   19,175,591.24 (±1,963,425.66)   
                      1.0   16,191,014.43 (±2,191,672.49)   
                      4.0   11,295,894.61 (±1,945,934.21)   
           sphere     0.5                   74.02 (±5.04)   
                      1.0                   59.04 (±4.85)   
                      4.0                   42.77 (±7.71)   

method                                       NNdrop_Worst  \
experiment function   freq                                  
exp1       rastrigin  0.5                   25.20 (±3.13)   
                      1.0                   22.21 (±2.89)   
                      4.0                   20.36 (±4.19)   
           rosenbrock 0.5        468,626.01 (±305,705.97)   
                      1.0        157,765.36 (±168,332.93)   
                      4.0         76,088.08 (±119,665.21)   
           sphere     0.5                   20.42 (±7.55)   
                      1.0                    9.58 (±7.55)   
                      4.0                   12.78 (±6.97)   
exp2       rastrigin  0.5                    2.91 (±0.52)   
                      1.0                    1.87 (±0.21)   
                      4.0                    1.05 (±0.08)   
           rosenbrock 0.5                 355.31 (±92.35)   
                      1.0                 114.44 (±46.30)   
                      4.0                   20.57 (±4.31)   
           sphere     0.5                    0.38 (±0.13)   
                      1.0                    0.20 (±0.06)   
                      4.0                    0.04 (±0.03)   
exp3       rastrigin  0.5                    7.62 (±2.69)   
                      1.0                   13.09 (±4.82)   
                      4.0                   29.33 (±1.67)   
           rosenbrock 0.5              3,953.16 (±735.66)   
                      1.0               4,190.50 (±32.91)   
                      4.0              4,069.83 (±768.23)   
           sphere     0.5                    5.22 (±0.07)   
                      1.0         

In [6]:
print(final_data.to_csv(float_format='%.2f'))

experiment,function,freq,NNdrop_Random,NNdrop_Worst,NNnorm_Random,NNnorm_Worst,noNNReval_mof.csv
exp1,rastrigin,0.50,24.14 (±3.93),25.20 (±3.13),16.40 (±5.93),17.02 (±6.77),20.24 (±0.06)
exp1,rastrigin,1.00,21.33 (±3.68),22.21 (±2.89),15.47 (±5.47),16.94 (±5.43),21.90 (±0.47)
exp1,rastrigin,4.00,19.29 (±6.28),20.36 (±4.19),11.75 (±4.87),11.48 (±5.23),17.91 (±0.13)
exp1,rosenbrock,0.50,"680,083.62 (±468,075.91)","468,626.01 (±305,705.97)","504,044.85 (±445,063.44)","483,430.93 (±404,002.00)","71,719.74 (±4,327.15)"
exp1,rosenbrock,1.00,"253,692.23 (±151,509.51)","157,765.36 (±168,332.93)","56,145.15 (±51,920.09)","84,796.47 (±81,515.00)","83,986.73 (±3,445.84)"
exp1,rosenbrock,4.00,"129,485.47 (±156,675.88)","76,088.08 (±119,665.21)","27,979.63 (±33,885.09)","37,506.97 (±39,308.06)","85,830.59 (±1,274.26)"
exp1,sphere,0.50,18.31 (±4.67),20.42 (±7.55),12.33 (±6.10),17.42 (±9.93),19.27 (±1.30)
exp1,sphere,1.00,13.27 (±6.87),9.58 (±7.55),5.80 (±3.67),7.47 (±5.11),23.37 (±0.61)
exp1,sphere,

In [26]:
!rm tmp.csv

rm: tmp.csv: No such file or directory


In [ ]:
Func_names=['sphere','rastrigin','rosenbrock']
exp_names=['exp1','exp2','exp3','exp4']
frequency=1000
nn_window=5


In [ ]:
#Experiment = Enum('Experiment', 'exp1 exp2 exp3 exp4')
#Method = Enum('Methods', 'noNNRestart noNNReval NNnorm NNdrop')
#Mechname:Random, Worst, Closest
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0
total=[]
#lables = ['noNNReval', 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']
lables = ['noNNReval', 'NNnorm_Worst', 'NNdrop_Worst']

for experiment in exp_names:
    
    for func in Func_names:
       # path = Path(f'../../data/results/{experiment}/{func}')
        path_in = Path(f'../../data/cluster_results/{experiment}/{func}')
        path = path_in / f'freq{frequency}nn_w{nn_window}nn_p{3}'
        noNNReval= pd.read_csv(path/f'noNNReval_{measure_name[index_measure]}.csv').mean(0).values[0]

        #noNNRestart= pd.read_csv(path/'noNNRestart_mof.csv')

       # NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


       # NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        #y=[noNNReval, NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        y=[noNNReval, NNnorm_Worst,NNdrop_Worst]

        total.append(y)
        #print(total)

    #,  index=[Func_names,Func_names]

#df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])
df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock', 'sphere','rastrigin','rosenbrock'])

#pd.options.display.float_format = '{:,.2f}'.format


print(pd.DataFrame.to_csv(df, float_format='%.2f'))


#pd.DataFrame.to_latex(df, multirow=True)    #, float_format="%%.2f" ,, , longtable=True,  bold_rows=True,


In [ ]:
pd.read_csv('tmp.csv')

In [ ]:
df

In [ ]:
#report nn_time
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=4
total=[]
lables = [ 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']

for experiment in exp_names:
    
    for func in Func_names:
        path = Path(f'../../data/results/{experiment}/{func}')

        NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]

        NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        y=[ NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        total.append(y)
        #print(total)


df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])



print(pd.DataFrame.to_csv(df, float_format='%.2f'))
#df
